In [1]:
import pandas as pd
pd.options.display.max_columns = None
# 671112131516
import os
import warnings
warnings.filterwarnings('ignore')
import datetime
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

In [2]:
path = '../dataofweek/'

In [3]:
#定义评价函数
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [4]:
#提取base_feature
def get_base_feature(df):
    df['time'] = df.START_DATE.apply(lambda x:x[:19])
    df['time'] = pd.to_datetime(df['time'],format='%Y-%m-%d-%H.%M.%S')
    timedelta = df['time'] - pd.datetime(df['time'].dt.year[0],df['time'].dt.month[0],df['time'].dt.day[0],0,0,0)
    df['30_min'] = timedelta.dt.seconds/1800
    df['30_min'] = df['30_min'].astype(int)
    df['20_min'] = timedelta.dt.seconds/1200
    df['20_min'] = df['20_min'].astype(int)
    df['10_min'] = timedelta.dt.seconds/600
    df['10_min'] = df['10_min'].astype(int)
    df['5_min'] = timedelta.dt.seconds/300
    df['5_min'] = df['5_min'].astype(int)

    df['hour'] = df.START_DATE.apply(lambda x:int(x[11:13]))
    
    df.TRADE_TYPE[df.TRADE_TYPE==21] = 1
    df.TRADE_TYPE[df.TRADE_TYPE==22] = 0
    
    result = df.groupby(['date','TRADE_ADDRESS','INDUSTRY_CODE','CARD_TYPE_EX','CARD_TYPE','SAM_ID','TERMINAL_ID','RECORD_ROW','30_min','20_min','10_min','5_min']).TRADE_TYPE.agg(['count','sum']).reset_index()
    result['inNums'] = result['sum']
    result['outNums'] = result['count'] - result['sum']
    del result['count'],result['sum']
    
    result.date = pd.to_datetime(result.date)
    result['week'] = result.date.dt.dayofweek+1
    del df
    return result

In [6]:
#根据EDA的分析进行数据的读取
data_list = ['12','13','14','15','16']
data1 = pd.DataFrame()
for i in data_list:
    file = str(i)+'_week.csv'
    df = pd.read_csv(path+file)
    data1 = pd.concat([data1,df],axis=0,ignore_index=True)         

In [7]:
le = LabelEncoder()

In [8]:
data1['TERMINAL_ID'] = le.fit_transform(data1['TERMINAL_ID'].values)

In [9]:
data1['SAM_ID'] = le.fit_transform(data1['SAM_ID'].values)

In [10]:
prevent = data1.copy()

In [11]:
del data1

In [12]:
#进一步提取特征
def more_feature(result):
    tmp = result.copy()

    tmp = result.groupby(['TRADE_ADDRESS'], as_index=False)['inNums'].agg({
                                                                        'inNums_ID_w_max'    : 'max',
                                                                        'inNums_ID_w_min'    : 'min', 
                                                                        'inNums_ID_w_mean'   : 'mean',
                                                                        'inNums_ID_w_sum'   : 'sum'
                                                                        })
    result = result.merge(tmp, on=['TRADE_ADDRESS'], how='left')

    tmp = result.groupby(['TRADE_ADDRESS','week','30_min'], as_index=False)['inNums'].agg({
                                                                        'inNums_ID_w_h_3_max'    : 'max',
                                                                        'inNums_ID_w_h_3_min'    : 'min', 
                                                                        'inNums_ID_w_h_3_mean'   : 'mean',
                                                                        'inNums_ID_w_h_3_sum'   : 'sum'
                                                                        })
    result = result.merge(tmp,on=['TRADE_ADDRESS','week','30_min'],how = 'left')
    #20
    tmp = result.groupby(['TRADE_ADDRESS','week','20_min'], as_index=False)['inNums'].agg({
                                                                        'inNums_ID_w_h_2_max'    : 'max',
                                                                        'inNums_ID_w_h_2_min'    : 'min', 
                                                                        'inNums_ID_w_h_2_mean'   : 'mean',
                                                                
                                                                        })
    result = result.merge(tmp,on=['TRADE_ADDRESS','week','20_min'],how = 'left')
    
    tmp = result.groupby(['TRADE_ADDRESS','week','5_min'], as_index=False)['inNums'].agg({
                                                                        'inNums_ID_w_h_5_max'    : 'max',
                                                                        'inNums_ID_w_h_5_min'    : 'min', 
                                                           
                                                                        'inNums_ID_w_h_5_sum'   : 'sum'
                                                                        })
    result = result.merge(tmp,on=['TRADE_ADDRESS','week','5_min'],how = 'left')
    
    
    
    ###出站与进站类似

    
    
    tmp = result.groupby(['TRADE_ADDRESS'], as_index=False)['outNums'].agg({
                                                                        'outNums_ID_max'    : 'max',
                                                                        'outNums_ID_min'    : 'min',
                                                                        'outNums_ID_mean'   : 'mean',
                                                                        'outNums_ID_sum'   : 'sum'
                                                                        })
    result = result.merge(tmp, on=['TRADE_ADDRESS'], how='left')
    

    tmp = result.groupby(['TRADE_ADDRESS','week','30_min'], as_index=False)['outNums'].agg({
                                                                       
                                                                        'outNums_ID_w_h_3_min'    : 'min', 
                                                           
                                                                      })
    result = result.merge(tmp,on=['TRADE_ADDRESS','week','30_min'],how = 'left')

    tmp = result.groupby(['TRADE_ADDRESS','week','20_min'], as_index=False)['outNums'].agg({
                                                              
                                                                        'outNums_ID_w_h_2_min'    : 'min', 
                                                    
                                         
                                                                        })
    result = result.merge(tmp,on=['TRADE_ADDRESS','week','20_min'],how = 'left')
    
    tmp = result.groupby(['TRADE_ADDRESS','week','5_min'], as_index=False)['outNums'].agg({
                                                  
                                                                        'outNums_ID_w_h_5_mean'   : 'mean',
                                               
                                                                        })
    result = result.merge(tmp,on=['TRADE_ADDRESS','week','5_min'],how = 'left')
    
    return result

In [13]:
data2 = prevent[(prevent.week!=5)&(prevent.week!=6)&(prevent.week!=7)]

In [14]:
data2 = get_base_feature(data2)

In [15]:
data9 = more_feature(data2)

In [16]:
data= data9.copy()

In [17]:
del data2,data9

In [18]:
data.shape

(9046398, 36)

In [19]:
data.head()

,date,TRADE_ADDRESS,INDUSTRY_CODE,CARD_TYPE_EX,CARD_TYPE,SAM_ID,TERMINAL_ID,RECORD_ROW,30_min,20_min,10_min,5_min,inNums,outNums,week,inNums_ID_w_max,inNums_ID_w_min,inNums_ID_w_mean,inNums_ID_w_sum,inNums_ID_w_h_3_max,inNums_ID_w_h_3_min,inNums_ID_w_h_3_mean,inNums_ID_w_h_3_sum,inNums_ID_w_h_2_max,inNums_ID_w_h_2_min,inNums_ID_w_h_2_mean,inNums_ID_w_h_5_max,inNums_ID_w_h_5_min,inNums_ID_w_h_5_sum,outNums_ID_max,outNums_ID_min,outNums_ID_mean,outNums_ID_sum,outNums_ID_w_h_3_min,outNums_ID_w_h_2_min,outNums_ID_w_h_5_mean
0,2015-10-20,121,0,0,98,32,6,4,14,21,42,85,0,1,2,1,0,0.548106,264642,1,0,0.560664,1890,1,0,0.485650,1,0,464,3,0,0.454027,219218,0,0,0.322628
1,2015-10-20,121,0,0,98,32,6,6,16,24,49,99,0,1,2,1,0,0.548106,264642,1,0,0.749665,4471,1,0,0.680522,0,0,0,3,0,0.454027,219218,0,0,1.000000
2,2015-10-20,121,0,0,98,32,6,7,16,24,49,99,0,1,2,1,0,0.548106,264642,1,0,0.749665,4471,1,0,0.680522,0,0,0,3,0,0.454027,219218,0,0,1.000000
3,2015-10-20,121,0,0,98,32,6,20,35,53,106,212,0,1,2,1,0,0.548106,264642,1,0,0.383571,2176,1,0,0.427173,1,0,687,3,0,0.454027,219218,0,0,0.474251
4,2015-10-20,121,0,0,98,32,6,21,37,55,111,222,0,1,2,1,0,0.548106,264642,1,0,0.503195,2205,1,0,0.420511,0,0,0,3,0,0.454027,219218,0,0,1.004396


In [20]:
#删除类别列别超过90%的列
cols = list(data.columns)
for col in cols:
    rate = data[col].value_counts(normalize = True,dropna = False).values[0]
    if ((rate>0.9)&(col!='id_week')):
        cols.remove(col)
        print(col,rate)
data = data[cols]

INDUSTRY_CODE 0.9222655249083669
inNums_ID_w_max 0.941837955836124
inNums_ID_w_h_3_max 0.9987367347755427
inNums_ID_w_h_2_max 0.9991168860799624
inNums_ID_w_h_5_min 0.9995647991609479
outNums_ID_min 1.0
outNums_ID_w_h_3_min 0.999818601834675


In [21]:
data['id_week'] = 0


data.id_week[(data.date>='2015-10-20')&(data.date<='2015-10-26')] = 1
data.id_week[(data.date>='2015-10-27')&(data.date<='2015-11-2')] = 2
data.id_week[(data.date>='2015-11-3')&(data.date<='2015-11-9')] = 3
data.id_week[(data.date>='2015-11-10')&(data.date<='2015-11-16')] = 4
data.id_week[(data.date>='2015-11-17')&(data.date<='2015-11-23')] = 5


In [22]:
sub = data[data.id_week==5]

In [23]:
#lgb参数
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2,
    'min_child_weight':6
}

In [24]:
columns = ['TRADE_ADDRESS',
 'CARD_TYPE_EX',
 'CARD_TYPE',
 'SAM_ID',
 'TERMINAL_ID',
 '5_min',
 'RECORD_ROW',
           'inNums',
           'outNums',
 'inNums_ID_w_mean',
 'inNums_ID_w_sum',
 'inNums_ID_w_h_3_mean',
 'inNums_ID_w_h_3_sum',
 'inNums_ID_w_h_2_mean',
 'outNums_ID_max',
 'outNums_ID_mean',
 'outNums_ID_sum',
 'inNums_ID_w_h_5_sum',
 'outNums_ID_w_h_5_mean']

In [25]:
#########inNums
# train_in = data[data.id_week!=4]
train_in = data[data.id_week<4]
train_in = train_in[columns]
y_train_in = train_in['inNums']

del train_in['outNums'],train_in['inNums']

valid = data[data.id_week==4]
valid = valid[columns]
y_valid = valid['inNums']
del valid['outNums'],valid['inNums']
# y_valid = np.log1p(y_valid)

test_in = data[data.id_week==5]
test_in = test_in[columns]
y_test_in = test_in['inNums']
del test_in['inNums'],test_in['outNums']

In [26]:
X_train_in = data[data.id_week<4]
X_train_in = X_train_in[columns]
X_y_train_in = X_train_in['inNums']

del X_train_in['outNums'],X_train_in['inNums']

In [27]:
lgb_train = lgb.Dataset(train_in, y_train_in)
lgb_evals = lgb.Dataset(valid, y_valid , reference=lgb_train)
gbm1 = lgb.train(params,
                lgb_train,
                num_boost_round=20000,
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                early_stopping_rounds=50,
                verbose_eval=1000,
                )
# lgb_train = lgb.Dataset(X_train_in, X_y_train_in)
# gbm1 = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=gbm1.best_iteration,
# #                 num_boost_round=20000,
#                 valid_sets=[lgb_train],
#                 valid_names=['train'],
#                 early_stopping_rounds=50,
#                 verbose_eval=1000,
#                 )

Training until validation scores don't improve for 50 rounds.
[1000]	train's l1: 0.0253315	valid's l1: 0.0255529
[2000]	train's l1: 0.0155573	valid's l1: 0.0158692
[3000]	train's l1: 0.0116776	valid's l1: 0.0121377
[4000]	train's l1: 0.00953908	valid's l1: 0.0101056
[5000]	train's l1: 0.00814504	valid's l1: 0.00878238
[6000]	train's l1: 0.00714369	valid's l1: 0.00781901
[7000]	train's l1: 0.00648697	valid's l1: 0.0071992
[8000]	train's l1: 0.00601067	valid's l1: 0.00676338
[9000]	train's l1: 0.00559718	valid's l1: 0.00638117
[10000]	train's l1: 0.00525862	valid's l1: 0.0060684
[11000]	train's l1: 0.00497429	valid's l1: 0.0058038
[12000]	train's l1: 0.00473595	valid's l1: 0.00558143
[13000]	train's l1: 0.00453886	valid's l1: 0.00540054
[14000]	train's l1: 0.00437356	valid's l1: 0.00525128
[15000]	train's l1: 0.00422357	valid's l1: 0.00511669
[16000]	train's l1: 0.00409211	valid's l1: 0.00499753
[17000]	train's l1: 0.00397472	valid's l1: 0.00489427
[18000]	train's l1: 0.00386994	valid's 

In [28]:
prediction_in = gbm1.predict(test_in, num_iteration=gbm1.best_iteration)

In [29]:
error_in = mean_absolute_error(np.round(prediction_in),y_test_in)
error_in

0.0004589598434636605

In [30]:
error_in = mean_absolute_error(prediction_in,y_test_in)
error_in

0.00569660819003189

In [31]:
sub['pre_inNums'] = prediction_in
#结果修正
# sub.loc[sub.pre_inNums<0,'pre_inNums'] = 0

In [32]:
mean_absolute_percentage_error(sub['pre_inNums'],y_test_in)

0.005696608190031503

In [33]:
#########inNums
# train_out = data[data.id_week!=4]
train_out = data[data.id_week<4]
train_out = train_out[columns]
y_train_out = train_out['outNums']
del train_out['outNums'],train_out['inNums']

valid = data[data.id_week==4]
valid = valid[columns]
y_valid = valid['outNums']
del valid['outNums'],valid['inNums']
# y_valid = np.log1p(y_valid)

test_out = data[data.id_week==5]
test_out = test_out[columns]
y_test_out = test_out['outNums']
del test_out['inNums'],test_out['outNums']

In [34]:
X_train_out = data[data.id_week<=4]
X_train_out = X_train_out[columns]
X_y_train_out = X_train_out['outNums']
del X_train_out['outNums'],X_train_out['inNums']

In [35]:
lgb_train = lgb.Dataset(train_out, y_train_out)
lgb_evals = lgb.Dataset(valid, y_valid , reference=lgb_train)
gbm2 = lgb.train(params,
                lgb_train,
                num_boost_round=50000,
                valid_sets=[lgb_train,lgb_evals],
                valid_names=['train','valid'],
                early_stopping_rounds=50,
                verbose_eval=1000,
                )
# lgb_train = lgb.Dataset(X_train_out, X_y_train_out)
# gbm2 = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=gbm2.best_iteration,
# #                 num_boost_round=50000,
#                 valid_sets=[lgb_train],
#                 valid_names=['train'],
#                 early_stopping_rounds=50,
#                 verbose_eval=1000,
#                 )

Training until validation scores don't improve for 50 rounds.
[1000]	train's l1: 0.0268493	valid's l1: 0.0270196
[2000]	train's l1: 0.0170717	valid's l1: 0.0174356
[3000]	train's l1: 0.0134479	valid's l1: 0.0139499
[4000]	train's l1: 0.0114117	valid's l1: 0.0120078
[5000]	train's l1: 0.010128	valid's l1: 0.0107736
[6000]	train's l1: 0.0092252	valid's l1: 0.0099045
[7000]	train's l1: 0.00856972	valid's l1: 0.0092804
[8000]	train's l1: 0.00809748	valid's l1: 0.00884146
[9000]	train's l1: 0.00773926	valid's l1: 0.00850577
[10000]	train's l1: 0.0074685	valid's l1: 0.00825664
[11000]	train's l1: 0.00724949	valid's l1: 0.00805376
[12000]	train's l1: 0.00708035	valid's l1: 0.00790208
[13000]	train's l1: 0.00692731	valid's l1: 0.00776446
[14000]	train's l1: 0.00680233	valid's l1: 0.00765234
[15000]	train's l1: 0.00669181	valid's l1: 0.00755542
[16000]	train's l1: 0.00659837	valid's l1: 0.0074736
[17000]	train's l1: 0.00651405	valid's l1: 0.00740108
[18000]	train's l1: 0.00643194	valid's l1: 0.

In [36]:
prediction_out = gbm2.predict(test_out, num_iteration=gbm2.best_iteration)
error_out = mean_absolute_error(np.round(prediction_out),y_test_out)
print('outNums的分数',error_out)

outNums的分数 0.0019381934077468937


In [37]:
features=train_out.columns
feature_rank=gbm1.feature_importance()
features_df=pd.DataFrame({ 'column':features,'importance': feature_rank}).sort_values(by='importance', ascending=False)
print(features_df)

                   column  importance
4             TERMINAL_ID      206742
16  outNums_ID_w_h_5_mean      197650
3                  SAM_ID      175466
6              RECORD_ROW      121791
9    inNums_ID_w_h_3_mean      120054
11   inNums_ID_w_h_2_mean       99620
5                   5_min       94324
15    inNums_ID_w_h_5_sum       88594
10    inNums_ID_w_h_3_sum       35063
2               CARD_TYPE       27203
0           TRADE_ADDRESS       25795
7        inNums_ID_w_mean       13957
8         inNums_ID_w_sum       12207
1            CARD_TYPE_EX       11129
14         outNums_ID_sum        4983
13        outNums_ID_mean        4733
12         outNums_ID_max         689


In [38]:
features=train_out.columns
feature_rank=gbm2.feature_importance()
features_df=pd.DataFrame({ 'column':features,'importance': feature_rank}).sort_values(by='importance', ascending=False)
print(features_df)

                   column  importance
6              RECORD_ROW      393117
4             TERMINAL_ID      320214
3                  SAM_ID      300051
16  outNums_ID_w_h_5_mean      271467
9    inNums_ID_w_h_3_mean      211469
11   inNums_ID_w_h_2_mean      196314
5                   5_min      195940
15    inNums_ID_w_h_5_sum      141978
10    inNums_ID_w_h_3_sum      124158
2               CARD_TYPE       35863
0           TRADE_ADDRESS       31505
1            CARD_TYPE_EX       28133
7        inNums_ID_w_mean       26417
8         inNums_ID_w_sum       23153
14         outNums_ID_sum       10705
13        outNums_ID_mean        8570
12         outNums_ID_max        1110


In [39]:
sub['pre_outNums'] = prediction_out
#结果修正
# sub.loc[sub.pre_outNums<0,'pre_outNums'] = 0

In [40]:
sub1 = sub.groupby(['date','TRADE_ADDRESS']).pre_inNums.agg({'inNums':'sum'}).reset_index()
sub2 = sub.groupby(['date','TRADE_ADDRESS']).pre_outNums.agg({'outNums':'sum'}).reset_index()
sub_result = sub1.merge(sub2,on = ['date','TRADE_ADDRESS'],how='left')
sub_result['flow'] = sub_result.inNums + sub_result.outNums
sub_result['round_pre'] = np.round(sub_result.flow)
sub_result['round_in'] = np.round(sub_result.inNums)
sub_result['round_out'] = np.round(sub_result.outNums)

In [41]:
real1 = sub.groupby(['date','TRADE_ADDRESS']).inNums.agg({'inNums':'sum'}).reset_index()
real2 = sub.groupby(['date','TRADE_ADDRESS']).outNums.agg({'outNums':'sum'}).reset_index()
real = real1.merge(real2,on = ['date','TRADE_ADDRESS'],how='left')
real['flow'] = real.inNums + real.outNums

In [42]:
error = mean_absolute_percentage_error(sub_result.flow,real.flow)
error

10.355598006306895

In [43]:
sub_result.head()

,date,TRADE_ADDRESS,inNums,outNums,flow,round_pre,round_in,round_out
0,2015-11-17,121,12506.204445,10450.615472,22956.819917,22957.0,12506.0,10451.0
1,2015-11-17,123,5393.123858,4975.428877,10368.552735,10369.0,5393.0,4975.0
2,2015-11-17,125,9937.377972,10811.640441,20749.018413,20749.0,9937.0,10812.0
3,2015-11-17,127,10138.285073,9885.660601,20023.945674,20024.0,10138.0,9886.0
4,2015-11-17,129,7421.426736,7053.162989,14474.589724,14475.0,7421.0,7053.0


In [52]:
list(sub_result.columns)

['date',
 'TRADE_ADDRESS',
 'inNums',
 'outNums',
 'flow',
 'round_pre',
 'round_in',
 'round_out']

In [53]:
sub_columns= ['date',
 'TRADE_ADDRESS',
 'round_pre',
 'round_in',
 'round_out']

In [54]:
sub_result = sub_result[sub_columns]

In [55]:
sub_result.head()

,date,TRADE_ADDRESS,round_pre,round_in,round_out
0,2015-11-17,121,22957.0,12506.0,10451.0
1,2015-11-17,123,10369.0,5393.0,4975.0
2,2015-11-17,125,20749.0,9937.0,10812.0
3,2015-11-17,127,20024.0,10138.0,9886.0
4,2015-11-17,129,14475.0,7421.0,7053.0


In [44]:
real.head()

,date,TRADE_ADDRESS,inNums,outNums,flow
0,2015-11-17,121,12526,10450,22976
1,2015-11-17,123,5398,4968,10366
2,2015-11-17,125,9947,10815,20762
3,2015-11-17,127,10156,9861,20017
4,2015-11-17,129,7431,7048,14479


In [56]:
re_path = '../result1/'
sub_result.to_csv(re_path+'1_2_3_4_flow_pre'+str('%.2f'%error)+'.csv',encoding='utf-8',index=False)
real.to_csv(re_path+'1_2_3_4_flow_real.csv',encoding='utf-8',index=False)

In [48]:
# #进行时间序列的交叉验证
# test_week = [4,5,6]
# # test_day = [23,24,25,28]
# error_in = []
# for i in test_week:
#     test = data[data.id_week== i+1]
#     y_test = test['inNums']
    
#     del test['id_week']
#     del test['inNums']
#     del test['outNums']
    
#     print('###############################inNums验证集',i)

#     train = data[data.id_week<i]
#     valid = data[data.id_week==i]
#     y_train = train['inNums']
#     y_valid = valid['inNums']
    
#     del train['inNums'],valid['inNums'],train['id_week']
#     del train['outNums'],valid['outNums'],valid['id_week']
    
#     lgb_train = lgb.Dataset(train,y_train)
#     lgb_evals  = lgb.Dataset(valid,y_valid,reference=lgb_train)
    
    
#     gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=10000,
#                 valid_sets=[lgb_train,lgb_evals],
#                 valid_names=['train','valid'],
#                 early_stopping_rounds=200,
#                 verbose_eval=1000,
#                 )
    
#     prediction_in = gbm.predict(test)
    

#     error = mean_absolute_percentage_error(np.abs(np.round(prediction_in)),y_test)
#     error_in.append(error)
#     print('验证集：',i)
#     print('验证集下一天作为测试集的误差为：',error)
        
# print('inNums的CV验证分数：',np.mean(error_in))

In [49]:
# #进行时间序列的交叉验证
# test_week = [4,5,6]
# # test_day = [23,24,25,28]
# error_in = []
# for i in test_week:
#     test = data[data.id_week== i+1]
#     y_test = test['outNums']
    
#     del test['id_week']
#     del test['inNums']
#     del test['outNums']
    
#     print('###############################outNums验证集',i)

#     train = data[data.id_week<i]
#     valid = data[data.id_week==i]
#     y_train = train['outNums']
#     y_valid = valid['outNums']
    
#     del train['inNums'],valid['inNums'],train['id_week']
#     del train['outNums'],valid['outNums'],valid['id_week']
    
#     lgb_train = lgb.Dataset(train,y_train)
#     lgb_evals  = lgb.Dataset(valid,y_valid,reference=lgb_train)
    
    
#     gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=10000,
#                 valid_sets=[lgb_train,lgb_evals],
#                 valid_names=['train','valid'],
#                 early_stopping_rounds=200,
#                 verbose_eval=1000,
#                 )
    
#     prediction_in = gbm.predict(test)
    

#     error = mean_absolute_percentage_error(np.abs(np.round(prediction_in)),y_test)
#     error_in.append(error)
#     print('验证集：',i)
#     print('验证集下一天作为测试集的误差为：',error)
        
# print('outNums的CV验证分数：',np.mean(error_in))

#log
inNums的CV验证分数： 2.6328033937801556
outNums的CV验证分数： 1.569054964432997
#######更改了参数
inNums的CV验证分数： 2.5018023015473205
outNums的CV验证分数： 1.5004474075321914
########